In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate
import json
%config Completer.use_jedi = False  # Fixes autocomplete issues
%config InlineBackend.figure_format = 'retina'  # Improves plot resolution

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp

In [ ]:
#-------------------------------
# data paths setup
#-------------------------------

#data_dir = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1')
data_dir = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3')
#data_dir = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day4')
rawdata_paths = [Path(p) for p in data_dir.iterdir() if p.is_dir() and not p.name.endswith('_processedData')]

#-------------------------------
# initial variables setup
#-------------------------------
rawdata_path = rawdata_paths[0]
time_window_start = -2  # s, FOR PLOTTING PURPOSES
time_window_end = 5  # s, FOR PLOTTING PURPOSES
baseline_window = (-2, 0) # s, FOR baselining averages 
event_name = "Apply halt: 2s" #FIXME No halt has broken somehow... 
vestibular_mismatch = False
common_resampled_rate = 1000 #in Hz
plot_fig1 = False
# for saccades
framerate = 59.77  # Hz (in the future, should come from saved data)
threshold = 65  # px/s FIXME make this adaptive
refractory_period = pd.Timedelta(milliseconds=100)  # msec, using pd.Timedelta for datetime index
plot_saccade_detection_QC = True

data_path = rawdata_path.parent / f"{rawdata_path.name}_processedData/downsampled_data"
save_path = rawdata_path.parent / f"{rawdata_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])


In [ ]:
#-------------------------------
# load downsampled data 
#-------------------------------
photometry_tracking_encoder_data = pd.read_parquet(data_path / "photometry_tracking_encoder_data.parquet", engine="pyarrow")
camera_photodiode_data = pd.read_parquet(data_path / "camera_photodiode_data.parquet", engine="pyarrow")
experiment_events = pd.read_parquet(data_path / "experiment_events.parquet", engine="pyarrow")
photometry_info = pd.read_parquet(data_path / "photometry_info.parquet", engine="pyarrow")
session_settings = pd.read_parquet(data_path / "session_settings.parquet", engine="pyarrow")
session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)

print(f"✅ Finished loading all parquet files")

# Calculate time differences between event_name events
event_times = experiment_events[experiment_events["Event"] == event_name].index
time_diffs = event_times.to_series().diff().dropna().dt.total_seconds()

# # Print the 5 shortest time differences
# print("5 shortest time differences between events:")
# print(time_diffs.nsmallest(5))

if (time_diffs < 10).any():
    print(f"⚠️ Warning: Some '{event_name}' events are less than 10 seconds apart. Consider applying a filter to events.")

mouse_name = process.check_exp_events(experiment_events, photometry_info, verbose = True)


In [ ]:
#---------------------------------------------------
# PLOT FIGURE to ascertain everything is well loaded
#---------------------------------------------------

df_to_analyze = photometry_tracking_encoder_data["Photodiode_int"] #using downsampled values in common time grid 
#df_to_analyze = camera_photodiode_data["Photodiode"] #use async raw values if needed for troubleshooting, but the nearest indices needs to be found , see couple of lines below

if vestibular_mismatch or event_name == "No halt": #determine halt times based on experiment events 
    photodiode_halts = experiment_events[experiment_events["Event"] == event_name].index.tolist()
    nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
    photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices] # as experiment events timestamps are not in the same time grid as downsampled data
    print ("ℹ️ INFO: vestibular MM or 'No halt', no signal in the photodiode, using experiment events for MM times")
else: #determine exact halt times based on photodiode signal
    photodiode_halts, photodiode_delay_min, photodiode_delay_avg, photodiode_delay_max = process.analyze_photodiode(df_to_analyze, experiment_events, event_name, plot = True)
# nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
# photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices]
if plot_fig1:
    process.plot_figure_1(photometry_tracking_encoder_data, session_name, save_path, common_resampled_rate, photodiode_halts, save_figure = False, show_figure = True, downsample_factor=50)
else: 
    print ("ℹ️ INFO: skipping figure 1")
del df_to_analyze
gc.collect()
None

## LOOK INTO THE superbig filtering in the velocity and ACC calculations (looks like it's 1 Hz???)

# detect saccades by hard threshold on velocity, the apply a 50 ms refractory period 

In [ ]:
# detect saccades 
df = photometry_tracking_encoder_data.copy()

# 1) Compute velocity (units/s). Filter/diff/filter. Because sample rate is 1000 Hz, diff is * 1000.
window_size = int(round(2 / framerate * 1000))
df["velocity"] = df["Ellipse.Center.X_eye1"].rolling(window=window_size, center=True, min_periods=1).mean()
df["velocity"] = df["velocity"].diff() * 1000
window_size = int(round(4 / framerate * 1000))
df["velocity"] = df["velocity"].rolling(window=window_size, center=True, min_periods=1).mean()

# 2) Define a velocity threshold for saccades (adjust as needed)
# implement adaptive filter in the future 

# 3) Create a boolean mask for samples exceeding the threshold
df["is_saccade"] = df["velocity"].abs() > threshold

# 4) Group consecutive saccade samples to form saccade events.
#    Label each contiguous "True" block with a unique ID.
df["saccade_id"] = (df["is_saccade"] & ~df["is_saccade"].shift(fill_value=False)).cumsum() * df["is_saccade"]

# 5) Extract saccade onset times and basic details for each saccade.
saccade_events = []
for sacc_id, group in df.groupby("saccade_id"):
    if sacc_id == 0:
        continue
    saccade_time = group.index[0]
    peak_time = group["velocity"].abs().idxmax()  # Save the time when the absolute velocity peaks
    peak_velocity = group["velocity"].abs().max()
    direction = "positive" if group["velocity"].mean() > 0 else "negative"
    
    saccade_events.append({
        "saccade_id": sacc_id,
        "saccade_time": saccade_time,
        "peak_time": peak_time,         # New column for peak time
        "peak_velocity": peak_velocity,
        "direction": direction
    })

# 6) Apply a refractory period of 50 ms: if 2 saccade events occur within 50 ms, keep only the first.
last_event_time = pd.Timestamp.min  # initialize with the earliest possible timestamp
filtered_saccade_events = []
for event in saccade_events:
    if event["saccade_time"] - last_event_time >= refractory_period:
        filtered_saccade_events.append(event)
        last_event_time = event["saccade_time"]

# 7) For each filtered saccade event, calculate the baseline and relative peak.
frame_duration = 1 / framerate  # seconds per frame
for event in filtered_saccade_events:
    saccade_time = event["saccade_time"]
    # Baseline: average the data for 3 frames immediately BEFORE the saccade onset.
    baseline_start = saccade_time - pd.Timedelta(seconds=3 * frame_duration)
    baseline = df.loc[baseline_start:saccade_time, "Ellipse.Center.X_eye1"].mean()
    event["baseline"] = baseline

    # Relative peak: in the next 40 ms after the saccade onset, measure the peak change relative to the baseline.
    window_end = saccade_time + pd.Timedelta(milliseconds=500)
    saccade_window = df.loc[saccade_time:window_end, "Ellipse.Center.X_eye1"]
    if event["direction"] == "positive":
        relative_peak = saccade_window.max() - baseline
    else:
        relative_peak = baseline - saccade_window.min()
    event["relative_peak"] = relative_peak

# Create a DataFrame of the filtered saccade events including the new metrics.
results_df = pd.DataFrame(filtered_saccade_events)


## It's a mess, below code is shit at peak detection, above code as well... ploting below is also messed up with variable names from the two different cells for peak detection

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

threshold_factor = 10  # Tweak to match your data

df = photometry_tracking_encoder_data.copy()

###############################################################################
# 1) Compute velocity (units/s) - same as your original approach
###############################################################################
window_size = int(round(2 / framerate * 1000))
df["velocity"] = df["Ellipse.Center.X_eye1"].rolling(window=window_size, center=True, min_periods=1).mean()
df["velocity"] = df["velocity"].diff() * 1000
window_size = int(round(4 / framerate * 1000))
df["velocity"] = df["velocity"].rolling(window=window_size, center=True, min_periods=1).mean()

###############################################################################
# 2) Define a robust threshold using median + MAD (adjust factor as needed)
###############################################################################
abs_vel = df["velocity"].abs()
median_vel = abs_vel.median()
# mad_vel = 1.4826 * (abs_vel - median_vel).abs().median()  # 1.4826 ~ scale factor for Gaussian
# velocity_threshold = median_vel + threshold_factor * mad_vel
velocity_threshold = 50 

###############################################################################
# 3) Find velocity peaks above that threshold
###############################################################################
peaks, properties = find_peaks(abs_vel, height=velocity_threshold)

###############################################################################
# 4) For each peak, determine onset and offset by stepping backward and forward
#    until velocity drops below a fraction of the threshold. Then find the
#    true position peak (max or min) within that onset–offset window.
###############################################################################
saccade_events = []
fraction_of_threshold = 0.05  # define onset/offset where |v| < velocity_threshold * fraction_of_threshold
n_baseline_frames = 3        # frames for baseline before onset

for peak_idx in peaks:
    # Velocity peak info
    peak_time = df.index[peak_idx]
    peak_velocity = df.loc[peak_time, "velocity"]  # can be positive or negative
    abs_peak_velocity = abs(peak_velocity)
    direction = "positive" if peak_velocity > 0 else "negative"
    
    half_thresh = velocity_threshold * fraction_of_threshold

    # -------------------------------
    # 4a) Find saccade onset
    #     Step backward from the velocity peak until velocity < half_thresh
    # -------------------------------
    onset_idx = peak_idx
    while onset_idx > 0 and abs_vel.iloc[onset_idx] > half_thresh:
        onset_idx -= 1
    
    saccade_onset_time = df.index[onset_idx]

    # -------------------------------
    # 4b) Find saccade offset
    #     Step forward from the velocity peak until velocity < half_thresh
    # -------------------------------
    offset_idx = peak_idx
    while offset_idx < len(df) - 1 and abs_vel.iloc[offset_idx] > half_thresh:
        offset_idx += 1
    
    saccade_offset_time = df.index[offset_idx]

    # -------------------------------
    # 4c) Baseline: average the pupil position for 'n_baseline_frames' before onset
    # -------------------------------
    baseline_start = max(0, onset_idx - n_baseline_frames)
    baseline_positions = df["Ellipse.Center.X_eye1"].iloc[baseline_start:onset_idx]
    if len(baseline_positions) > 0:
        baseline_position = baseline_positions.mean()
    else:
        baseline_position = df["Ellipse.Center.X_eye1"].iloc[0]  # fallback

    # -------------------------------
    # 4d) Find the TRUE position peak within onset–offset
    #     If direction is positive, look for max position; if negative, look for min
    # -------------------------------
    pos_segment = df["Ellipse.Center.X_eye1"].iloc[onset_idx : offset_idx + 1]

    if direction == "positive":
        # Find maximum position
        true_peak_idx = pos_segment.idxmax()
    else:
        # Find minimum position
        true_peak_idx = pos_segment.idxmin()

    true_peak_time = true_peak_idx
    true_peak_position = df.loc[true_peak_time, "Ellipse.Center.X_eye1"]

    # -------------------------------
    # 4e) Compute amplitude
    # -------------------------------
    amplitude = true_peak_position - baseline_position

    # -------------------------------
    # Store event
    # -------------------------------
    saccade_events.append({
        "velocity_peak_idx": peak_idx,
        "velocity_peak_time": peak_time,
        "velocity_peak_value": peak_velocity,
        "saccade_onset_time": saccade_onset_time,
        "saccade_offset_time": saccade_offset_time,
        "true_peak_time": true_peak_time,
        "true_peak_position": true_peak_position,
        "direction": direction,
        "abs_peak_velocity": abs_peak_velocity,
        "amplitude": amplitude
    })

###############################################################################
# 5) (Optional) Refractory period filter (e.g. 50 ms)
###############################################################################
saccade_events_sorted = sorted(saccade_events, key=lambda x: x["velocity_peak_time"])
filtered_saccade_events = []
refractory_period = pd.Timedelta("50ms")
last_saccade_time = None

for ev in saccade_events_sorted:
    if last_saccade_time is None or (ev["velocity_peak_time"] - last_saccade_time) >= refractory_period:
        filtered_saccade_events.append(ev)
        last_saccade_time = ev["velocity_peak_time"]

# Convert to DataFrame
saccade_events_df = pd.DataFrame(filtered_saccade_events)


In [ ]:
# if plot_saccade_detection_QC:
#     pio.renderers.default = 'browser'
    
#     fig = go.Figure()
#     # Plot the continuous Ellipse.Center.X_eye1 data
#     fig.add_trace(go.Scatter(
#         x=df.index,
#         y=df["Ellipse.Center.X_eye1"],
#         mode="lines",
#         name="Ellipse.Center.X_eye1",
#         line=dict(color="darkgrey", width=1)
#     ))
    
#     if not results_df.empty:
#         # Separate positive and negative saccade events
#         pos_df = results_df[results_df["direction"] == "positive"]
#         neg_df = results_df[results_df["direction"] == "negative"]
        
#         # Positive saccade onsets (circle-open markers, red)
#         fig.add_trace(go.Scatter(
#             x=pos_df["saccade_onset_time"],
#             y=df.loc[pos_df["saccade_onset_time"], "Ellipse.Center.X_eye1"],
#             mode="markers",
#             marker=dict(symbol="circle-open", size=10, line=dict(width=2, color="red")),
#             name="Positive Saccade Onset"
#         ))
        
#         # Negative saccade onsets (circle-open markers, blue)
#         fig.add_trace(go.Scatter(
#             x=neg_df["saccade_onset_time"],
#             y=df.loc[neg_df["saccade_onset_time"], "Ellipse.Center.X_eye1"],
#             mode="markers",
#             marker=dict(symbol="circle-open", size=10, line=dict(width=2, color="blue")),
#             name="Negative Saccade Onset"
#         ))
        
#         # Positive saccade peaks (filled circle, red) -- uses 'true_peak_time'
#         fig.add_trace(go.Scatter(
#             x=pos_df["true_peak_time"],
#             y=df.loc[pos_df["true_peak_time"], "Ellipse.Center.X_eye1"],
#             mode="markers",
#             marker=dict(symbol="circle", size=10, color="red"),
#             name="Positive Saccade Peak"
#         ))
        
#         # Negative saccade peaks (filled circle, blue) -- uses 'true_peak_time'
#         fig.add_trace(go.Scatter(
#             x=neg_df["true_peak_time"],
#             y=df.loc[neg_df["true_peak_time"], "Ellipse.Center.X_eye1"],
#             mode="markers",
#             marker=dict(symbol="circle", size=10, color="blue"),
#             name="Negative Saccade Peak"
#         ))
    
#     fig.update_layout(
#         title="Ellipse Center X with Saccade Markers",
#         xaxis_title="Time",
#         yaxis_title="Ellipse.Center.X_eye1"
#     )
#     fig.show()


In [ ]:
# NOT WORKING WELL - saccade peak detection 

# if plot_saccade_detection_QC:
#     pio.renderers.default = 'browser'
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(
#         x=df.index, 
#         y=df["Ellipse.Center.X_eye1"], 
#         mode="lines", 
#         name="Ellipse.Center.X_eye1", 
#         line=dict(color="darkgrey", width=1), 
#         yaxis="y2"
#     ))
#     if not results_df.empty:
#         pos_df = results_df[results_df["direction"]=="positive"]
#         neg_df = results_df[results_df["direction"]=="negative"]
#         if not pos_df.empty:
#             pos_peaks = pos_df["peak_time"]
#             pos_y = df.loc[pos_peaks, "Ellipse.Center.X_eye1"]
#             fig.add_trace(go.Scatter(
#                 x=pos_peaks, 
#                 y=pos_y, 
#                 mode="markers",
#                 marker=dict(symbol="circle-open", size=10, line=dict(width=2, color="red")),
#                 name="Positive Saccade Peaks", 
#                 yaxis="y2"
#             ))
#         if not neg_df.empty:
#             neg_peaks = neg_df["peak_time"]
#             neg_y = df.loc[neg_peaks, "Ellipse.Center.X_eye1"]
#             fig.add_trace(go.Scatter(
#                 x=neg_peaks, 
#                 y=neg_y, 
#                 mode="markers",
#                 marker=dict(symbol="circle-open", size=10, line=dict(width=2, color="blue")),
#                 name="Negative Saccade Peaks", 
#                 yaxis="y2"
#             ))
#     fig.update_layout(
#         title="Velocity and Ellipse.Center.X_eye1",
#         xaxis_title="Time",
#         yaxis=dict(title="Velocity"),
#         yaxis2=dict(title="Ellipse.Center.X_eye1", overlaying="y", side="right")
#     )
#     fig.show()


In [ ]:
# #plot the distribution of saccade amplitudes and inter-saccade intervals 

# pio.renderers.default = 'notebook'

# pos_df = results_df[results_df["direction"]=="positive"]
# neg_df = results_df[results_df["direction"]=="negative"]
# # neg_amplitudes = neg_df["relative_peak"] * -1
# max_amp = max(pos_df["relative_peak"].max() if not pos_df.empty else 0,
#               neg_df["relative_peak"].max() if not neg_df.empty else 0)
# sorted_times = results_df.sort_values("saccade_time")["saccade_time"]
# intervals_ms = sorted_times.diff().dropna().dt.total_seconds() * 1000

# fig = make_subplots(rows=1, cols=2, subplot_titles=["Saccade Amplitude Histogram", "Inter-Saccade Interval Histogram"])
# fig.add_trace(go.Histogram(x=pos_df["relative_peak"], nbinsx=30, opacity=0.6, name="Positive Saccades", marker_color="red"), row=1, col=1)
# fig.add_trace(go.Histogram(x=neg_amplitudes, nbinsx=30, opacity=0.6, name="Negative Saccades", marker_color="blue"), row=1, col=1)
# fig.update_xaxes(title_text="Saccade Amplitude", range=[-max_amp, max_amp], row=1, col=1)
# fig.update_yaxes(title_text="Count", row=1, col=1)
# auto_bins = np.histogram_bin_edges(intervals_ms, bins='auto'); nbins_auto = len(auto_bins) - 1
# fig.add_trace(go.Histogram(x=intervals_ms, nbinsx=nbins_auto, opacity=0.75, name="Inter-Saccade Intervals", marker_color="green"), row=1, col=2)
# fig.update_xaxes(title_text="Interval (ms)", row=1, col=2)
# fig.update_yaxes(title_text="Count", type="log", row=1, col=2)
# fig.update_layout(title="Saccade Analysis: Amplitude and Inter-Saccade Intervals", barmode="overlay", template="plotly_white", width=1000, height=500)
# fig.show()


### Plot all aligned data for individual halt events, the plot baselined averages ± sem

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp
import math
import pandas as pd
import gc

# --- Data Alignment ---
aligned_data = []

for halt_time in photodiode_halts:
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time + pd.Timedelta(seconds=time_window_start)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=time_window_end))
    ].copy()
    
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    window_data["Halt Time"] = halt_time
    aligned_data.append(window_data)

aligned_df = pd.concat(aligned_data, ignore_index=True)

# --- Subplot Grid Setup ---
n_events = len(photodiode_halts)
n_cols = 4
n_rows = math.ceil(n_events / n_cols)

# Create subplots with a single (default) y-axis in each cell.
specs = [[{} for _ in range(n_cols)] for _ in range(n_rows)]
subplot_titles = [f'Event: {halt_time}' for halt_time in photodiode_halts]
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=subplot_titles, specs=specs)

# We'll add an extra y-axis for the ellipse center (X) traces.
# base_extra is used to create unique axis IDs starting after the auto-assigned primary axes.
base_extra = n_events + 1

for i, halt_time in enumerate(photodiode_halts):
    row = (i // n_cols) + 1
    col = (i % n_cols) + 1
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    
    # -- Fluorescence Traces on Primary y-axis --
    fig.add_trace(
        go.Scatter(
            x=subset["Time (s)"],
            y=subset["z_470"],
            mode='lines',
            name='z_470',
            line=dict(color='green')
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=subset["Time (s)"],
            y=subset["z_560"],
            mode='lines',
            name='z_560',
            line=dict(color='red')
        ),
        row=row, col=col
    )
    
    # -- Extra y-axis for Ellipse Center X Trace (only) --
    extra_axis_number = base_extra + i
    extra_axis_trace_ref = f"y{extra_axis_number}"
    
    fig.add_trace(
        go.Scatter(
            x=subset["Time (s)"],
            y=subset["Ellipse.Center.X_eye1"],
            mode='lines',
            name='Ellipse Center X',
            line=dict(color='purple')  # solid line (default)
        ),
        row=row, col=col
    )
    # Update the trace to use the extra y-axis.
    fig.data[-1].update(yaxis=extra_axis_trace_ref)
    
    # --- Define the Extra y-axis for this Subplot in the Layout ---
    # Determine the subplot's x-axis anchor.
    xaxis_number = (row - 1) * n_cols + col
    x_anchor = "x" if xaxis_number == 1 else f"x{xaxis_number}"
    # The primary y-axis for this subplot: for the first subplot it's "y", then "y2", "y3", etc.
    primary_y = "y" if xaxis_number == 1 else f"y{xaxis_number}"
    
    extra_axis_layout_key = f"yaxis{extra_axis_number}"
    fig.layout[extra_axis_layout_key] = dict(
        title="Ellipse Center X",
        overlaying=primary_y,
        anchor=x_anchor,
        side="right",
        position=0.95,  # Must be within [0,1]
        showgrid=False
    )

# --- Update Common Axis Labels ---
fig.update_xaxes(title_text="Time (s)")
fig.update_yaxes(title_text="Fluorescence (z-score)")

fig.update_layout(
    height=400 * n_rows,
    width=350 * n_cols,
    title_text="Fluorescence & Ellipse Center X for Each Event",
    template='plotly_white'
)

fig.show()

del aligned_data
gc.collect()


In [ ]:
# Create an empty DataFrame to store aligned data
aligned_data = []

# Define the selected indices
#selected_indices = [0, 1, 11, 16]
selected_indices = list(range(len(photodiode_halts)))

# Create an empty list to store aligned data
aligned_data = []

# Loop through each halt event time using enumerate
for idx, halt_time in enumerate(photodiode_halts):
    if idx not in selected_indices:
        continue  # Skip halt times that are not in our selected indices
    
    # Extract data within the selected time window
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time + pd.Timedelta(seconds=time_window_start)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=time_window_end))
    ].copy()

    # Compute time relative to halt
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    
    # Add event identifier
    window_data["Halt Time"] = halt_time

    # Store aligned data
    aligned_data.append(window_data)

# Concatenate all windows into a single DataFrame
aligned_df = pd.concat(aligned_data, ignore_index=True)


# Compute mean and standard error of the mean (SEM)
mean_df = aligned_df.groupby("Time (s)").mean()
sem_df = aligned_df.groupby("Time (s)").sem()  

# Create figure for the two plots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True)

### PLOT 1: Individual Traces - Photodiode, z_470, z_560 ###
ax1 = axes[0]

for halt_time in photodiode_halts:
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    ax1.plot(subset["Time (s)"], subset["Photodiode_int"], color='grey', alpha=0.5)

ax1.set_xlabel('Time (s) relative to halt')
ax1.set_ylabel('Photodiode')
ax1.set_title('Photodiode, z_470, and z_560')

ax1_2 = ax1.twinx()
for halt_time in photodiode_halts:
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    ax1_2.plot(subset["Time (s)"], subset["z_470"], color='green', linestyle='-', alpha=0.5)
    ax1_2.plot(subset["Time (s)"], subset["z_560"], color='red', linestyle='-', alpha=0.5)

ax1_2.set_ylabel('Fluorescence (z-score)', color='green')

### PLOT 2: Mean & SEM of All Signals ###
ax2 = axes[1]

# Photodiode
ax2.plot(mean_df.index, mean_df["Photodiode_int"], color='grey', alpha=0.8)
ax2.fill_between(mean_df.index, mean_df["Photodiode_int"] - sem_df["Photodiode_int"], 
                 mean_df["Photodiode_int"] + sem_df["Photodiode_int"], color='grey', alpha=0.2)

ax2.set_xlabel('Time (s) relative to halt')
ax2.set_ylabel('Photodiode')
ax2.set_title('Mean & SEM of All Signals')

# Fluorescence signals (z_470 and z_560)
ax2_2 = ax2.twinx()
ax2_2.plot(mean_df.index, mean_df["z_470"], color='green', linestyle='-', alpha=0.8)
ax2_2.fill_between(mean_df.index, mean_df["z_470"] - sem_df["z_470"], 
                   mean_df["z_470"] + sem_df["z_470"], color='green', alpha=0.2)

ax2_2.plot(mean_df.index, mean_df["z_560"], color='red', linestyle='-', alpha=0.8)
ax2_2.fill_between(mean_df.index, mean_df["z_560"] - sem_df["z_560"], 
                   mean_df["z_560"] + sem_df["z_560"], color='red', alpha=0.2)

ax2_2.set_ylabel('Fluorescence (z-score)', color='green')

# Motor Velocity
ax2_3 = ax2.twinx()
ax2_3.spines['right'].set_position(('outward', 50))  
ax2_3.plot(mean_df.index, mean_df["Motor_Velocity"], color='#00008B', linestyle='-', alpha=0.8)
ax2_3.fill_between(mean_df.index, mean_df["Motor_Velocity"] - sem_df["Motor_Velocity"], 
                   mean_df["Motor_Velocity"] + sem_df["Motor_Velocity"], color='#00008B', alpha=0.2)
ax2_3.set_ylabel('Motor Velocity (Dark Blue)')
ax2_3.yaxis.label.set_color('#00008B')

# Running Velocity (Velocity_0X)
ax2_4 = ax2.twinx()
ax2_4.spines['right'].set_position(('outward', 100))  
ax2_4.plot(mean_df.index, mean_df["Velocity_0X"]*1000, color='orange', linestyle='-', alpha=0.8)
ax2_4.fill_between(mean_df.index, (mean_df["Velocity_0X"] - sem_df["Velocity_0X"])*1000, 
                   (mean_df["Velocity_0X"] + sem_df["Velocity_0X"])*1000, color='orange', alpha=0.2)
ax2_4.set_ylabel('Running velocity (mm/s²) WRONG SCALE?', color='orange')

# Turning Velocity (Velocity_0Y)
ax2_5 = ax2.twinx()
ax2_5.spines['right'].set_position(('outward', 150))  
ax2_5.plot(mean_df.index, mean_df["Velocity_0Y"], color='#4682B4', linestyle='-', alpha=0.8)
ax2_5.fill_between(mean_df.index, mean_df["Velocity_0Y"] - sem_df["Velocity_0Y"], 
                   mean_df["Velocity_0Y"] + sem_df["Velocity_0Y"], color='#4682B4', alpha=0.2)
ax2_5.set_ylabel('Turning velocity (deg/s²) WRONG SCALE?', color='#4682B4')

# ---- New code added below for Ellipse signals ----

# Ellipse.Diameter
ax2_6 = ax2.twinx()
ax2_6.spines['right'].set_position(('outward', 200))  
ax2_6.plot(mean_df.index, mean_df["Ellipse.Diameter_eye1"], color='purple', linestyle='-', alpha=0.8)
ax2_6.fill_between(mean_df.index, mean_df["Ellipse.Diameter_eye1"] - sem_df["Ellipse.Diameter_eye1"], 
                   mean_df["Ellipse.Diameter_eye1"] + sem_df["Ellipse.Diameter_eye1"], color='purple', alpha=0.2)
ax2_6.set_ylabel('Pupil Diameter', color='purple')

# Ellipse.Center.X
ax2_7 = ax2.twinx()
ax2_7.spines['right'].set_position(('outward', 250))  
ax2_7.plot(mean_df.index, mean_df["Ellipse.Center.X_eye1"], color='magenta', linestyle='-', alpha=0.8)
ax2_7.fill_between(mean_df.index, mean_df["Ellipse.Center.X_eye1"] - sem_df["Ellipse.Center.X_eye1"], 
                   mean_df["Ellipse.Center.X_eye1"] + sem_df["Ellipse.Center.X_eye1"], color='magenta', alpha=0.2)
ax2_7.set_ylabel('Ellipse Center X', color='magenta')

# Adjust layout and show plot
fig.tight_layout()
plt.show()

del aligned_data
gc.collect()
None



In [ ]:
photodiode_halts

In [ ]:
# Set the plot width (in inches)
plot_width = 14  # Change this value to adjust the plot width

# Compute baseline values for each signal (excluding Photodiode)
baseline_df = aligned_df[(aligned_df["Time (s)"] >= baseline_window[0]) & 
                         (aligned_df["Time (s)"] <= baseline_window[1])].groupby("Halt Time").mean()

# Subtract baseline from each signal (except Photodiode)
for signal_name in ["z_470", "z_560", "Motor_Velocity", "Velocity_0X", "Velocity_0Y"]:
    aligned_df[f"{signal_name}_Baseline"] = aligned_df[signal_name] - aligned_df["Halt Time"].map(baseline_df[signal_name])

# ---- NEW: Baseline correction for Ellipse signals ----
for signal_name in ["Ellipse.Diameter_eye1", "Ellipse.Center.X_eye1"]:
    aligned_df[f"{signal_name}_Baseline"] = aligned_df[signal_name] - aligned_df["Halt Time"].map(baseline_df[signal_name])

# Compute mean and SEM of baseline-adjusted signals
mean_baseline_df = aligned_df.groupby("Time (s)").mean()
sem_baseline_df = aligned_df.groupby("Time (s)").sem()

# Function to ensure zero is centered while covering SEM values
def get_symmetric_ylim(mean_data, sem_data):
    max_abs_value = max(abs(mean_data).max() + sem_data.max(), abs(mean_data).min() - sem_data.min())
    return (-max_abs_value, max_abs_value)

# Create figure for the new baseline-corrected plot using plot_width parameter
fig, ax = plt.subplots(figsize=(plot_width, 6))

### PLOT: Mean & SEM of Baseline-Corrected Signals ###
ax.plot(mean_baseline_df.index, mean_baseline_df["Photodiode_int"], color='grey', alpha=0.8)
ax.fill_between(mean_baseline_df.index, mean_baseline_df["Photodiode_int"] - sem_baseline_df["Photodiode_int"], 
                mean_baseline_df["Photodiode_int"] + sem_baseline_df["Photodiode_int"], color='grey', alpha=0.2)

ax.set_xlabel('Time (s) relative to halt')
ax.set_ylabel('Photodiode', color='grey')
ax.set_title(f'Baselined Mean & SEM of All Signals - {mouse_name}')

# Fluorescence axis (z_470 and z_560)
ax2 = ax.twinx()
ax2.plot(mean_baseline_df.index, mean_baseline_df["z_470_Baseline"], color='green', linestyle='-', alpha=0.8)
ax2.fill_between(mean_baseline_df.index, mean_baseline_df["z_470_Baseline"] - sem_baseline_df["z_470_Baseline"], 
                 mean_baseline_df["z_470_Baseline"] + sem_baseline_df["z_470_Baseline"], color='green', alpha=0.2)
ax2.plot(mean_baseline_df.index, mean_baseline_df["z_560_Baseline"], color='red', linestyle='-', alpha=0.8)
ax2.fill_between(mean_baseline_df.index, mean_baseline_df["z_560_Baseline"] - sem_baseline_df["z_560_Baseline"], 
                 mean_baseline_df["z_560_Baseline"] + sem_baseline_df["z_560_Baseline"], color='red', alpha=0.2)
ax2.set_ylabel('Fluorescence (z-score, red 560nm)', color='green')
ax2.set_ylim(get_symmetric_ylim(
    pd.concat([mean_baseline_df["z_470_Baseline"], mean_baseline_df["z_560_Baseline"]]),
    pd.concat([sem_baseline_df["z_470_Baseline"], sem_baseline_df["z_560_Baseline"]])
))
ax2.yaxis.label.set_color('green')

# Motor Velocity axis
ax3 = ax.twinx()
ax3.spines['right'].set_position(('outward', 50))  
ax3.plot(mean_baseline_df.index, mean_baseline_df["Motor_Velocity_Baseline"], color='#00008B', linestyle='-', alpha=0.8)
ax3.fill_between(mean_baseline_df.index, mean_baseline_df["Motor_Velocity_Baseline"] - sem_baseline_df["Motor_Velocity_Baseline"], 
                 mean_baseline_df["Motor_Velocity_Baseline"] + sem_baseline_df["Motor_Velocity_Baseline"], color='#00008B', alpha=0.2)
ax3.set_ylabel('Motor Velocity (deg/s²)', color='#00008B')
ax3.set_ylim(get_symmetric_ylim(mean_baseline_df["Motor_Velocity_Baseline"], sem_baseline_df["Motor_Velocity_Baseline"]))
ax3.yaxis.label.set_color('#00008B')

# Running Velocity axis (Velocity_0X)
ax4 = ax.twinx()
ax4.spines['right'].set_position(('outward', 100))  
ax4.plot(mean_baseline_df.index, mean_baseline_df["Velocity_0X_Baseline"]*1000, color='orange', linestyle='-', alpha=0.8)
ax4.fill_between(mean_baseline_df.index, (mean_baseline_df["Velocity_0X_Baseline"] - sem_baseline_df["Velocity_0X_Baseline"])*1000, 
                 (mean_baseline_df["Velocity_0X_Baseline"] + sem_baseline_df["Velocity_0X_Baseline"])*1000, color='orange', alpha=0.2)
ax4.set_ylabel('Running velocity (mm/s²) WRONG SCALE?', color='orange')
ax4.set_ylim(get_symmetric_ylim(mean_baseline_df["Velocity_0X_Baseline"]*1000, sem_baseline_df["Velocity_0X_Baseline"]*1000))
ax4.yaxis.label.set_color('orange')

# Turning Velocity axis (Velocity_0Y)
ax5 = ax.twinx()
ax5.spines['right'].set_position(('outward', 150))  
ax5.plot(mean_baseline_df.index, mean_baseline_df["Velocity_0Y_Baseline"], color='#4682B4', linestyle='-', alpha=0.8)
ax5.fill_between(mean_baseline_df.index, mean_baseline_df["Velocity_0Y_Baseline"] - sem_baseline_df["Velocity_0Y_Baseline"], 
                 mean_baseline_df["Velocity_0Y_Baseline"] + sem_baseline_df["Velocity_0Y_Baseline"], color='#4682B4', alpha=0.2)
ax5.set_ylabel('Turning velocity (deg/s²) WRONG SCALE?', color='#4682B4')
ax5.set_ylim(get_symmetric_ylim(mean_baseline_df["Velocity_0Y_Baseline"], sem_baseline_df["Velocity_0Y_Baseline"]))
ax5.yaxis.label.set_color('#4682B4')

# ---- NEW: Plot Ellipse signals ----

# Ellipse.Diameter_eye1 axis
ax6 = ax.twinx()
ax6.spines['right'].set_position(('outward', 200))  
ax6.plot(mean_baseline_df.index, mean_baseline_df["Ellipse.Diameter_eye1_Baseline"], color='purple', linestyle='-', alpha=0.8)
ax6.fill_between(mean_baseline_df.index, mean_baseline_df["Ellipse.Diameter_eye1_Baseline"] - sem_baseline_df["Ellipse.Diameter_eye1_Baseline"], 
                mean_baseline_df["Ellipse.Diameter_eye1_Baseline"] + sem_baseline_df["Ellipse.Diameter_eye1_Baseline"], color='purple', alpha=0.2)
ax6.set_ylabel('Ellipse Diameter', color='purple')
ax6.set_ylim(get_symmetric_ylim(mean_baseline_df["Ellipse.Diameter_eye1_Baseline"], sem_baseline_df["Ellipse.Diameter_eye1_Baseline"]))

# Ellipse.Center.X_eye1 axis
ax7 = ax.twinx()
ax7.spines['right'].set_position(('outward', 250))  
ax7.plot(mean_baseline_df.index, mean_baseline_df["Ellipse.Center.X_eye1_Baseline"], color='magenta', linestyle='-', alpha=0.8)
ax7.fill_between(mean_baseline_df.index, mean_baseline_df["Ellipse.Center.X_eye1_Baseline"] - sem_baseline_df["Ellipse.Center.X_eye1_Baseline"], 
                mean_baseline_df["Ellipse.Center.X_eye1_Baseline"] + sem_baseline_df["Ellipse.Center.X_eye1_Baseline"], color='magenta', alpha=0.2)
ax7.set_ylabel('Ellipse Center X', color='magenta')
ax7.set_ylim(get_symmetric_ylim(mean_baseline_df["Ellipse.Center.X_eye1_Baseline"], sem_baseline_df["Ellipse.Center.X_eye1_Baseline"]))

# Adjust layout and show plot
fig.tight_layout()
plt.show()
# Save the figure as a PNG file at 1200 dpi
fig.savefig(save_path / f"figure2_{event_name}.png", dpi=1200, bbox_inches='tight')


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_selected_events(
    aligned_df,
    events_to_include,
    baseline_window,
    mouse_name,
    event_name,
    save_path,
    plot_width=14
):
    """
    Plots the mean & SEM of baseline-corrected signals for only the specified events.

    Parameters
    ----------
    aligned_df : pd.DataFrame
        DataFrame containing columns ["Time (s)", "Halt Time"] and the signals to be plotted.
    events_to_include : list of int
        List of specific event IDs (in "Halt Time") to include, e.g. [0, 1, 11, 16, 20].
    baseline_window : tuple
        A time window (start, end) in seconds for computing the baseline, e.g. (-1, 0).
    mouse_name : str
        Name of the mouse for the plot title.
    event_name : str
        Label for the event, used in the figure filename.
    save_path : Path or str
        Where to save the figure.
    plot_width : float
        Width of the plot in inches.
    """

    # 1) Subset data to only the events of interest
    subset_df = aligned_df[aligned_df["Halt Time"].isin(events_to_include)].copy()
    if subset_df.empty:
        print("No rows found for the specified events:", events_to_include)
        return

    # 2) Compute baseline values for each signal within the baseline_window
    #    *only* for the selected events
    baseline_mask = (
        (subset_df["Time (s)"] >= baseline_window[0]) &
        (subset_df["Time (s)"] <= baseline_window[1])
    )
    baseline_df = subset_df[baseline_mask].groupby("Halt Time").mean()

    # 3) Subtract baseline from each signal (except Photodiode)
    for signal_name in ["z_470", "z_560", "Motor_Velocity", "Velocity_0X", "Velocity_0Y"]:
        subset_df[f"{signal_name}_Baseline"] = (
            subset_df[signal_name] - 
            subset_df["Halt Time"].map(baseline_df[signal_name])
        )

    # (NEW) Baseline correction for Ellipse signals
    for signal_name in ["Ellipse.Diameter_eye1", "Ellipse.Center.X_eye1"]:
        subset_df[f"{signal_name}_Baseline"] = (
            subset_df[signal_name] -
            subset_df["Halt Time"].map(baseline_df[signal_name])
        )

    # 4) Compute mean and SEM for the baseline-adjusted signals
    mean_baseline_df = subset_df.groupby("Time (s)").mean()
    sem_baseline_df = subset_df.groupby("Time (s)").sem()

    # Helper function to get symmetrical y-limits around zero
    def get_symmetric_ylim(mean_data, sem_data):
        max_abs_value = max(
            abs(mean_data).max() + sem_data.max(),
            abs(mean_data).min() - sem_data.min()
        )
        return (-max_abs_value, max_abs_value)

    # 5) Create the figure
    fig, ax = plt.subplots(figsize=(plot_width, 6))

    # Photodiode
    ax.plot(mean_baseline_df.index, mean_baseline_df["Photodiode_int"], color='grey', alpha=0.8)
    ax.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Photodiode_int"] - sem_baseline_df["Photodiode_int"], 
        mean_baseline_df["Photodiode_int"] + sem_baseline_df["Photodiode_int"], 
        color='grey', alpha=0.2
    )
    ax.set_xlabel('Time (s) relative to halt')
    ax.set_ylabel('Photodiode', color='grey')
    ax.set_title(f'Baselined Mean & SEM of Selected Events - {mouse_name}')

    # Fluorescence axis (z_470 and z_560)
    ax2 = ax.twinx()
    ax2.plot(mean_baseline_df.index, mean_baseline_df["z_470_Baseline"], color='green', linestyle='-', alpha=0.8)
    ax2.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["z_470_Baseline"] - sem_baseline_df["z_470_Baseline"], 
        mean_baseline_df["z_470_Baseline"] + sem_baseline_df["z_470_Baseline"], 
        color='green', alpha=0.2
    )
    ax2.plot(mean_baseline_df.index, mean_baseline_df["z_560_Baseline"], color='red', linestyle='-', alpha=0.8)
    ax2.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["z_560_Baseline"] - sem_baseline_df["z_560_Baseline"], 
        mean_baseline_df["z_560_Baseline"] + sem_baseline_df["z_560_Baseline"], 
        color='red', alpha=0.2
    )
    ax2.set_ylabel('Fluorescence (z-score, red 560nm)', color='green')
    ax2.set_ylim(get_symmetric_ylim(
        pd.concat([mean_baseline_df["z_470_Baseline"], mean_baseline_df["z_560_Baseline"]]),
        pd.concat([sem_baseline_df["z_470_Baseline"], sem_baseline_df["z_560_Baseline"]])
    ))
    ax2.yaxis.label.set_color('green')

    # Motor Velocity axis
    ax3 = ax.twinx()
    ax3.spines['right'].set_position(('outward', 50))  
    ax3.plot(mean_baseline_df.index, mean_baseline_df["Motor_Velocity_Baseline"], color='#00008B', linestyle='-', alpha=0.8)
    ax3.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Motor_Velocity_Baseline"] - sem_baseline_df["Motor_Velocity_Baseline"], 
        mean_baseline_df["Motor_Velocity_Baseline"] + sem_baseline_df["Motor_Velocity_Baseline"], 
        color='#00008B', alpha=0.2
    )
    ax3.set_ylabel('Motor Velocity (deg/s²)', color='#00008B')
    ax3.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Motor_Velocity_Baseline"], 
        sem_baseline_df["Motor_Velocity_Baseline"]
    ))
    ax3.yaxis.label.set_color('#00008B')

    # Running Velocity axis (Velocity_0X)
    ax4 = ax.twinx()
    ax4.spines['right'].set_position(('outward', 100))  
    ax4.plot(mean_baseline_df.index, mean_baseline_df["Velocity_0X_Baseline"]*1000, color='orange', linestyle='-', alpha=0.8)
    ax4.fill_between(
        mean_baseline_df.index,
        (mean_baseline_df["Velocity_0X_Baseline"] - sem_baseline_df["Velocity_0X_Baseline"])*1000, 
        (mean_baseline_df["Velocity_0X_Baseline"] + sem_baseline_df["Velocity_0X_Baseline"])*1000, 
        color='orange', alpha=0.2
    )
    ax4.set_ylabel('Running velocity (mm/s²) WRONG SCALE?', color='orange')
    ax4.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Velocity_0X_Baseline"]*1000, 
        sem_baseline_df["Velocity_0X_Baseline"]*1000
    ))
    ax4.yaxis.label.set_color('orange')

    # Turning Velocity axis (Velocity_0Y)
    ax5 = ax.twinx()
    ax5.spines['right'].set_position(('outward', 150))  
    ax5.plot(mean_baseline_df.index, mean_baseline_df["Velocity_0Y_Baseline"], color='#4682B4', linestyle='-', alpha=0.8)
    ax5.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Velocity_0Y_Baseline"] - sem_baseline_df["Velocity_0Y_Baseline"], 
        mean_baseline_df["Velocity_0Y_Baseline"] + sem_baseline_df["Velocity_0Y_Baseline"], 
        color='#4682B4', alpha=0.2
    )
    ax5.set_ylabel('Turning velocity (deg/s²) WRONG SCALE?', color='#4682B4')
    ax5.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Velocity_0Y_Baseline"], 
        sem_baseline_df["Velocity_0Y_Baseline"] 
    ))
    ax5.yaxis.label.set_color('#4682B4')

    # Ellipse.Diameter_eye1 axis
    ax6 = ax.twinx()
    ax6.spines['right'].set_position(('outward', 200))  
    ax6.plot(mean_baseline_df.index, mean_baseline_df["Ellipse.Diameter_eye1_Baseline"], color='purple', linestyle='-', alpha=0.8)
    ax6.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Ellipse.Diameter_eye1_Baseline"] - sem_baseline_df["Ellipse.Diameter_eye1_Baseline"], 
        mean_baseline_df["Ellipse.Diameter_eye1_Baseline"] + sem_baseline_df["Ellipse.Diameter_eye1_Baseline"], 
        color='purple', alpha=0.2
    )
    ax6.set_ylabel('Ellipse Diameter', color='purple')
    ax6.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Ellipse.Diameter_eye1_Baseline"], 
        sem_baseline_df["Ellipse.Diameter_eye1_Baseline"]
    ))

    # Ellipse.Center.X_eye1 axis
    ax7 = ax.twinx()
    ax7.spines['right'].set_position(('outward', 250))  
    ax7.plot(mean_baseline_df.index, mean_baseline_df["Ellipse.Center.X_eye1_Baseline"], color='magenta', linestyle='-', alpha=0.8)
    ax7.fill_between(
        mean_baseline_df.index,
        mean_baseline_df["Ellipse.Center.X_eye1_Baseline"] - sem_baseline_df["Ellipse.Center.X_eye1_Baseline"], 
        mean_baseline_df["Ellipse.Center.X_eye1_Baseline"] + sem_baseline_df["Ellipse.Center.X_eye1_Baseline"], 
        color='magenta', alpha=0.2
    )
    ax7.set_ylabel('Ellipse Center X', color='magenta')
    ax7.set_ylim(get_symmetric_ylim(
        mean_baseline_df["Ellipse.Center.X_eye1_Baseline"], 
        sem_baseline_df["Ellipse.Center.X_eye1_Baseline"]
    ))

    fig.tight_layout()
    plt.show()
    # Save the figure
    fig.savefig(save_path / f"figure2_{event_name}.png", dpi=1200, bbox_inches='tight')


# ----------------------------
# Example usage:
# ----------------------------
events_to_include = [0, 1, 11, 16, 20]
plot_selected_events(
    aligned_df=aligned_df,
    events_to_include=events_to_include,
    baseline_window=(-1, 0),  # example baseline window
    mouse_name="MouseA",
    event_name="MyEvents",
    save_path=Path("./output_plots"),
    plot_width=14
)


In [ ]:
aligned_df.head()

In [ ]:
import importlib #for force updating changed packages 
# Force reload the modules
importlib.reload(harp_resources.process)
importlib.reload(harp_resources.utils)
# Reassign after reloading to ensure updated references
process = harp_resources.process
utils = harp_resources.utils

In [ ]:
pympler_memory_df = utils.get_pympler_memory_usage(top_n=10)
mouse_name = "B6J2717"